[View in Colaboratory](https://colab.research.google.com/github/ZackPashkin/Simply-python-script-for-sending-text-data-to-Natural-Language-API-and-BigQuery/blob/master/Simply_python_script_for_sending_text_data_to_Natural_Language_API_and_BigQuery.ipynb)

**What is it good for?**

*    sending the text data to the Natural Language API
*    classifying articles
*    importing articles to BigQuery


Create a file called *classify-text.py* and copy the following into it. 
Replace YOUR_PROJECT with your GCP Project ID.

In [0]:
from google.cloud import storage, language, bigquery

# Set up our GCS, NL, and BigQuery clients
storage_client = storage.Client()
nl_client = language.LanguageServiceClient()
# just replace YOUR_PROJECT with your project id below 
# it should look like this: bq_client = bigquery.Client(project='gcp-f1d55e5e42f12d87')
bq_client = bigquery.Client(project='YOUR_PROJECT')

dataset_ref = bq_client.dataset('news_classification_dataset')
dataset = bigquery.Dataset(dataset_ref)
table_ref = dataset.table('article_data') # Update this if you used a different table name
table = bq_client.get_table(table_ref)

# Send article text to the NL API's classifyText method
def classify_text(article):
        response = nl_client.classify_text(
                document=language.types.Document(
                        content=article,
                        type=language.enums.Document.Type.PLAIN_TEXT
                )
        )
        return response

rows_for_bq = []
files = storage_client.bucket('text-classification-codelab').list_blobs()
print("Got article files from GCS, sending them to the NL API (this will take ~2 minutes)...")

# Send files to the NL API and save the result to send to BigQuery
for file in files:
        if file.name.endswith('txt'):
                article_text = file.download_as_string()
                nl_response = classify_text(article_text)
                if len(nl_response.categories) > 0:
                        rows_for_bq.append((article_text, nl_response.categories[0].name, nl_response.categories[0].confidence))

print("Writing NL API article data to BigQuery...")
# Write article text + category data to BQ
errors = bq_client.insert_rows(table, rows_for_bq)
assert errors == []

 And run the following script:
 
** python classify-text.py**


What script is actually doing:

We're using the google-cloud Python client library to access Cloud Storage, the Natural Language API, and BigQuery.
 .  When classifying each article is done, the data is inserted into BigQuery using insert_rows().

*   First, a client is created for each service.
*   Then references are created to the BigQuery table.
*     *files* is a reference to each of the dataset files in the public bucket.
*   We iterate through these files, download the articles as strings, and send each one to the Natural Language API in our classify_text function.
*   For all articles where the Natural Language API returns a category, the article and its category data are saved to a rows_for_bq list.
*   When classifying each article is done, the data is inserted into BigQuery using insert_rows().



**Note:**
*The Natural Language API can return more than one category for a document, but for this script you're only storing the first category returned, to keep things simple.*

**Enjoy the script!**